# Park Recommender System Pipeline
Includes K-means, activity filter, and attendance filter functions

#### Loading packages and data

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import folium 
import calendar
import sklearn
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score

from collections import Counter

import yellowbrick
from yellowbrick.cluster import KElbowVisualizer
import ast

import chart_studio
import chart_studio.plotly as py
import plotly
import plotly.express as px
import plotly.figure_factory as ff
import numpy as np

pd.options.mode.chained_assignment = None

In [6]:
species = pd.read_csv('assets/all_species_112222.csv')
species.drop(species.columns[species.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

locations = pd.read_csv('assets/locations_112222.csv')
locations.drop(locations.columns[locations.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

# fixing NPSA coords
locations.loc[locations['Park Code'] == 'VIIS', 'Longitude'] = locations.loc[locations['Park Code'] == 'VIIS', 'Longitude']*-1

weather = pd.read_csv('assets/NPS_weather_trends_112222.csv')

activities = pd.read_csv('assets/activities.csv')
activities['cleaned'] = [ast.literal_eval(activities['cleaned'].iloc[i]) for i in range(len(activities))]

In [7]:
def regioncolors(counter):
    if counter['k_cluster'] == 0:
        return 'darkblue'
    elif counter['k_cluster'] == 1:
        return 'darkred'
    elif counter['k_cluster'] == 2:
        return 'blue'
    elif counter['k_cluster'] == 3:
        return 'darkgreen'
    else:
        return 'darkpurple'

### Transform Data and Cluster

In [31]:
def cluster(park, month, sim_or_diff):
    '''Input user park and month, return silhouette score, map, and parks in cluster'''
    
    park_weather = weather[['ParkName', 'Month', 'Year', 'Temp_Avg_Fahrenheit', 'Prcp_Avg_Inches']]

    # get 10-year averages for each month for each park
    avg_10_yr = park_weather.groupby(['ParkName','Month']).agg('mean').reset_index().drop(columns = 'Year')
    avg_10_yr.rename(columns = {'ParkName':'park', 'Month':'month','Temp_Avg_Fahrenheit':'temp', 'Prcp_Avg_Inches':'prcp'}, inplace = True)
    avg_10_yr['month_name'] = avg_10_yr['month'].apply(lambda x: calendar.month_abbr[x])
    avg_10_yr = avg_10_yr[avg_10_yr['month_name'] == month]

    clus_temp = avg_10_yr[['temp', 'prcp']]
    X = StandardScaler().fit_transform(clus_temp)

    # sort park names for future merging
    temp_merged = avg_10_yr.merge(locations, how = 'left', left_on = 'park', right_on = 'Park Name')
    sort_parks = temp_merged['Park Code'].tolist()
    # new df with only species that are present 
    present_sp = species[species['Occurrence'] == 'Present']
    similarity_df = present_sp[['Park Name', 'Scientific Name', 'Park Code']]
    sort_i = dict(zip(sort_parks, range(len(sort_parks))))
    similarity_df['park_code_ranked'] = similarity_df['Park Code'].map(sort_i)
    similarity_df.sort_values(by = ['park_code_ranked'], inplace = True)
    similarity_df.drop('park_code_ranked', axis = 1, inplace = True)

    # list of all park codes
    park_codes = list(similarity_df['Park Code'].unique())
    # list of all species in a park
    code_sp_list = []
    for code in park_codes:
        sp = list(similarity_df[similarity_df['Park Code'] == code]['Scientific Name'])
        #sp.append(code)
        code_sp_list.append(sp)

    # new park-species dataframe
    park_sp_df = pd.DataFrame()
    park_sp_df['park_code'] = park_codes
    park_sp_df['species_list'] = code_sp_list

    # 0 if species is not present, 1 if species is present
    mlb = MultiLabelBinarizer()
    vec = mlb.fit_transform(park_sp_df['species_list'])
    vecs = pd.DataFrame(vec, columns=mlb.classes_)

    # apply cosine_similarity fxn on df
    df_cosine = pd.DataFrame(cosine_similarity(vecs,dense_output=True))

    # pca for dimensitonality reduction
    pca = PCA(n_components =4)
    transform = pca.fit_transform(df_cosine)

    # concat weather and species vecs
    weather_sp = []
    for i in range(len(park_sp_df)):
        concat = np.concatenate((X[i], transform[i]))
        weather_sp.append(concat)
    weather_sp_arr = np.array(weather_sp)
    #print(weather_sp_arr.shape)
    visualizer = KElbowVisualizer(KMeans(random_state = 42), k=(2,11), show = False)
    visualizer.fit(weather_sp_arr)
    optimal_k = visualizer.elbow_value_

    KM = KMeans(n_clusters = optimal_k, random_state = 42)
    temp_labels = KM.fit_predict(weather_sp_arr)
    plt.close()
    labs = np.unique(temp_labels)
    #print(len(labs))
    avg_10_yr['k_cluster'] = temp_labels
    temp_merged = avg_10_yr.merge(locations, how = 'left', left_on = 'park', right_on = 'Park Name')

    location_list = temp_merged[['Latitude', 'Longitude']].values.tolist()
    park_names = temp_merged['Park Name'].tolist()

            
    temp_merged['color'] = temp_merged.apply(regioncolors, axis = 1)

    us_map = folium.Map(tiles='CartoDB positron', zoom_start=14)

    for point in range(0, len(location_list)):
        folium.Marker(location_list[point],popup=park_names[point], icon=folium.Icon(color=temp_merged["color"][point], icon_color='white', icon='star', angle=0, prefix='fa')).add_to(us_map)

    sil_score = silhouette_score(weather_sp_arr, KM.fit_predict(weather_sp_arr)) # a good silhouette score should be > 0.5
    #print(sil_score)
    user_cluster = temp_merged[temp_merged['park'] == park]['k_cluster'].item()
    user_parks = temp_merged[temp_merged['k_cluster'] == user_cluster]['park'].tolist()

    if sim_or_diff == 'Similar':
        user_parks = temp_merged[temp_merged['k_cluster'] == user_cluster]['park'].tolist()
    elif sim_or_diff == 'Different':
        user_parks = temp_merged[temp_merged['k_cluster'] != user_cluster]['park'].tolist()
    
    return user_parks, weather_sp_arr, temp_merged, us_map

In [32]:
cluster('Yosemite National Park', 'Jun', 'Similar')[3]

### Filter Clustered Parks w/ Activities

In [33]:
# highlight most similar parks
def park_col(counter):
    '''Change color of park markers'''
    if counter['score'] == 3:
        return 'blue'
    else:
        return 'gray'

In [34]:
def highlight_col(x):
    '''Changes the color of park rows that match all 3 activities'''
    df = x.copy()
    #set by condition
    mask = df['Activity Matches'] == 3
    df.loc[mask, :] = 'background-color: lightblue'
    df.loc[~mask,:] = 'background-color: ""'
    return df    

In [40]:
def activities_filter(lst_activities, lst_cluster, park):
    '''Input user activties, results from cluster fxn, user park, return final park map, activities df'''
    act_df = activities.merge(locations, how = 'left', left_on = 'park', right_on = 'Park Code')[['park', 'Park Name','cleaned']]
    user_acts = act_df[act_df['Park Name'].isin(lst_cluster)]

    # rank parks based on count of user activities 
    scores = []
    for i in range(len(user_acts)):
        if len(set(lst_activities).intersection(set(user_acts['cleaned'].iloc[i]))) == 3:
            scores.append(3)
        elif len(set(lst_activities).intersection(set(user_acts['cleaned'].iloc[i]))) == 2:
            scores.append(2)
        elif len(set(lst_activities).intersection(set(user_acts['cleaned'].iloc[i]))) == 1:
            scores.append(1)
        else:
            scores.append(0)
    
    user_acts['score'] = scores
    sorted_acts = user_acts.sort_values(by = 'score', ascending = False)
    # if user park is in final filtered df, drop row
    sorted_acts = sorted_acts[sorted_acts['Park Name'].str.contains(park) == False]

    act_df = sorted_acts.iloc[:5]
    act_df.rename(columns = {'cleaned':'Activities List'},inplace= True)
    act_df.reset_index(inplace = True, drop = True)

    act_loc_df = act_df.merge(locations, left_on = 'Park Name', right_on = 'Park Name')
    act_loc_df['color'] = act_loc_df.apply(park_col, axis = 1)
    
    location_list = act_loc_df[['Latitude', 'Longitude']].values.tolist()
    park_names = act_loc_df['Park Name'].tolist()

    # display final user map and final activities df
    result_map = folium.Map(tiles='CartoDB positron', zoom_start=14)
    for point in range(0, len(location_list)):
        folium.Marker(location_list[point],popup=park_names[point], 
        icon=folium.Icon(color=act_loc_df["color"][point], icon_color='white', icon='star', angle=0, prefix='fa')).add_to(result_map)
    #folium.map.LayerControl('topleft', collapsed = False).add_to(result_map)
    display(result_map)
    act_df.drop(columns = 'park', inplace = True)
    act_df.rename(columns = {'score':'Activity Matches'}, inplace = True)
    act_df['Park Name'] = act_df['Park Name'].str.replace('National Park', 'NP')

    # order activities where user activities are first in list 
    for i in range(len(act_df)):
        park_activities = act_df['Activities List'].iloc[i]
        for act in lst_activities:
            if act in park_activities:
                park_activities.insert(0, act)

        act_df['Activities List'].iloc[i] = list(dict.fromkeys(park_activities))
    act_df = act_df[['Park Name', 'Activity Matches', 'Activities List']]
    return act_df.style.apply(highlight_col, axis = None), act_df


In [36]:
def attendance_filter(park_list, popularity, sort_by_attendance = False, drop_percentage = 0.33):
    park_order_dict = {'Great Smoky Mountains National Park': 1,'Grand Canyon National Park': 2,'Yosemite National Park': 3,'Yellowstone National Park': 4,'Rocky Mountain National Park': 5,'Zion National Park': 6,'Olympic National Park': 7,'Grand Teton National Park': 8,'Acadia National Park': 9,'Cuyahoga Valley National Park': 10,'Glacier National Park': 11,'Indiana Dunes National Park': 12,'Joshua Tree National Park': 13,'Bryce Canyon National Park': 14,'Hawaii Volcanoes National Park': 15,'Hot Springs National Park': 16,'Shenandoah National Park': 17,'Mount Rainier National Park': 18,'Arches National Park': 19,'New River Gorge National Park and Preserve': 20,'Haleakala National Park': 21,'Death Valley National Park': 22,'Sequoia National Park': 23,'Everglades National Park': 24,'Badlands National Park': 25,'Capitol Reef National Park': 26,'Saguaro National Park': 27,'Petrified Forest National Park': 28,'Theodore Roosevelt National Park': 29,'Mammoth Cave National Park': 30,'Wind Cave National Park': 31,'Kings Canyon National Park': 32,'Canyonlands National Park': 33,'Crater Lake National Park': 34,'Biscayne National Park': 35,'Mesa Verde National Park': 36,'White Sands National Park': 37,'Denali National Park': 38,'Glacier Bay National Park': 39,'Lassen Volcanic National Park': 40,'Redwood National Park': 41,'Virgin Islands National Park': 42,'Carlsbad Caverns National Park': 43,'Big Bend National Park': 44,'Great Sand Dunes National Park': 45,'Channel Islands National Park': 46,'Kenai Fjords National Park': 47,'Voyageurs National Park': 48,'Black Canyon of the Gunnison National Park': 49,'Pinnacles National Park': 50,'Guadalupe Mountains National Park': 51,'Congaree National Park': 52,'Great Basin National Park': 53,'Wrangell - St Elias National Park': 54,'Dry Tortugas National Park': 55,'Katmai National Park': 56,'North Cascades National Park': 57,'Isle Royale National Park': 58,'National Park of American Samoa': 59,'Lake Clark National Park': 60,'Gates Of The Arctic National Park': 61,'Kobuk Valley National Park': 62}

    less_busy_parks ={}
    for key, value in park_order_dict.items():
        if value > 15:
            less_busy_parks[key] = value

    less_busy_parks_list = list(less_busy_parks.keys())
 
    full_park_list = list(park_order_dict.keys())

    if popularity == 'I do not mind crowds':
        ordered_input_parks = full_park_list

    edit_list = []
    if popularity == 'Yes':
        for park in park_list:
            if park in less_busy_parks_list:
                edit_list.append(park)
        ordered_input_parks = edit_list

    drop_n_parks = int(len(park_list) * drop_percentage) + 1
    ordered_input_parks = ordered_input_parks[:len(ordered_input_parks)-drop_n_parks]

    if sort_by_attendance:
        return ordered_input_parks
    else:
        park_list = [park for park in park_list if park in ordered_input_parks]
        return park_list

### Testing 
Show results for Yosemite NP, June, similar park, fishing/hiking/photography, doesn't mind crowds.

In [42]:
park_name = 'Yosemite National Park'
month = 'Jun'
sim_or_diff = 'Similar'
list_activities = ['Fishing', 'Hiking', 'Photography']
cluster_results = cluster(park_name, month, sim_or_diff)[0]

attendance_results = attendance_filter(cluster_results, 'I do not mind crowds')
activities_filter(list_activities, attendance_results, park_name)[0]

,Park Name,Activity Matches,Activities List
0,Great Sand Dunes NP,3,"['Photography', 'Hiking', 'Fishing', 'Backpacking', 'Stargazing', 'Horseback Riding', 'Biking']"
1,Rocky Mountain NP,3,"['Photography', 'Hiking', 'Fishing', 'Camping', 'Wildlife Viewing']"
2,Lassen Volcanic NP,2,"['Hiking', 'Fishing', 'Scenic Drives', 'Boating', 'Stargazing', 'Camping']"
3,Great Basin NP,2,"['Hiking', 'Fishing']"
4,Grand Canyon NP,2,"['Photography', 'Hiking', 'Backpacking', 'Scenic Drives', 'Canyoneering', 'Stargazing', 'Backcountry Camping', 'Biking', 'Wildlife Viewing']"


### Evaluation

In [132]:
user_df = activities_filter(list_activities, attendance_results, park_name)[1]
plt.close()

### Precision, Recall and F1-Score

In [133]:
act_df = activities.merge(locations, how = 'left', left_on = 'park', right_on = 'Park Code')[['park', 'Park Name','cleaned']]

# rank parks based on count of user activities 
scores = []
for i in range(len(act_df)):
    if len(set(list_activities).intersection(set(act_df['cleaned'].iloc[i]))) == 3:
        scores.append(3)
    elif len(set(list_activities).intersection(set(act_df['cleaned'].iloc[i]))) == 2:
        scores.append(2)
    elif len(set(list_activities).intersection(set(act_df['cleaned'].iloc[i]))) == 1:
        scores.append(1)
    else:
        scores.append(0)

act_df['score'] = scores

In [135]:
user_rel = user_df[user_df['Activity Matches'] >= 2]
user_rel

,Park Name,Activity Matches,Activities List
0,Great Sand Dunes NP,3,"[Photography, Hiking, Fishing, Backpacking, St..."
1,Rocky Mountain NP,3,"[Photography, Hiking, Fishing, Camping, Wildli..."
2,Lassen Volcanic NP,2,"[Hiking, Fishing, Scenic Drives, Boating, Star..."
3,Great Basin NP,2,"[Hiking, Fishing]"
4,Grand Canyon NP,2,"[Photography, Hiking, Backpacking, Scenic Driv..."


In [165]:
all_rel = act_df[act_df['score'] >= 2]

Precision and Recalll @ 5

In [129]:
precision = len(user_rel)/len(user_df)
precision

1.0

In [130]:
recall = len(user_rel)/len(all_rel)
recall

0.1724137931034483

In [131]:
F1 = 2*(precision*recall)/(precision + recall)
F1

0.29411764705882354

In [141]:
scores_dict = {}
for i in range(0, 4):
    
    user_rel = user_df[user_df['Activity Matches'] >= i]
    all_rel = act_df[act_df['score'] >= i]
    precision =  len(user_rel)/len(user_df)
    recall = len(user_rel)/len(all_rel)
    F1 = 2*(precision*recall)/(precision + recall)
    scores_dict[i] = [precision, recall, F1]

In [162]:
metrics_df = pd.DataFrame.from_dict(scores_dict, orient = 'index').reset_index()
metrics_df.rename(columns = {'index': 'threshold', 0: 'precision', 1:'recall', 2:'f1'}, inplace = True)

In [163]:
def highlight(row):
    if row.threshold == 2:
        return ['background-color: yellow'] * len(row)
    else:
        return ['background-color: white'] * len(row)

In [164]:
metrics_df.style.apply(highlight, axis = 1)

,threshold,precision,recall,f1
0,0,1.000000,0.080645,0.149254
1,1,1.000000,0.086207,0.158730
2,2,1.000000,0.172414,0.294118
3,3,0.400000,0.285714,0.333333
